In [1]:
import pandas as pd
import numpy as np
import time, warnings
import datetime as dt

warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv('analysis_rf_model.csv')
df = pd.DataFrame(df)
df['date'] = pd.to_datetime(df['date'])
df['last'] = pd.to_datetime(df['last'])
df.info()

#Add 7 hours to the data to convert it to bkk time
df['last'] +=  pd.to_timedelta(7, unit='h')
df.tail()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 173001 entries, 0 to 173000
Data columns (total 6 columns):
mid            173001 non-null int64
date           173001 non-null datetime64[ns]
last           173001 non-null datetime64[ns]
desktop        173001 non-null int64
mobile         173001 non-null int64
total_trans    173001 non-null int64
dtypes: datetime64[ns](2), int64(4)
memory usage: 7.9 MB


,mid,date,last,desktop,mobile,total_trans
172996,824085,2018-07-15 17:00:00,2018-07-16 00:31:37.495,0,4,4
172997,684155,2018-07-15 17:00:00,2018-07-16 00:31:02.906,0,7,7
172998,258844,2018-07-15 17:00:00,2018-07-16 00:31:02.578,5,0,5
172999,4575551,2018-07-15 17:00:00,2018-07-16 00:30:07.872,0,4,4
173000,596753,2018-07-15 17:00:00,2018-07-16 00:29:06.582,4,0,4


In [3]:
a = df['last'].max() -  df['last'].min()
a

Timedelta('6 days 23:30:52.515000')

In [4]:
df = df.sort_values('mid')
df = df.sort_values('last')
print(df.tail())

# Last date = 2018-07-22 23:59:59 -- we will use it as reference
now = dt.date(2018, 7, 22)
print(now)

# Create column just the date of 'last'
df['date_last'] = pd.DatetimeIndex(df['last']).date

       mid                date                    last  desktop  mobile  \
4   169062 2018-07-21 17:00:00 2018-07-22 23:59:58.891        0      27   
3  3266149 2018-07-21 17:00:00 2018-07-22 23:59:58.903       23       4   
2   366933 2018-07-21 17:00:00 2018-07-22 23:59:58.972        5       0   
1  1326831 2018-07-21 17:00:00 2018-07-22 23:59:59.089        0      19   
0  2465416 2018-07-21 17:00:00 2018-07-22 23:59:59.097        8       0   

   total_trans  
4           27  
3           27  
2            5  
1           19  
0            8  
2018-07-22


In [5]:
df = df.sort_values('mid')
df_recency = df.groupby(by='mid', as_index=False)['date_last'].max()
df_recency.columns = ['mid','last_online']


CALCULATING RECENCY

In [6]:
# calculate recency :(last day - last purchase)
df_recency['Recency'] = df_recency['last_online'].apply(lambda x: (now - x).days)
df_recency = df_recency[['mid', 'Recency']]
df_recency.head()


,mid,Recency
0,125,2
1,210,2
2,444,2
3,666,4
4,789,2


CALCULATING FREQEUNCY

Frequency helps us to know how many times a customer purchased from us. To do that we need to check how many transactions users have made.

In [7]:
df_freq = df.groupby(by='mid', as_index=False)['total_trans'].sum()
df_freq.columns = ['mid', 'Frequency']
df_freq.head()

,mid,Frequency
0,125,235
1,210,88
2,444,73
3,666,44
4,789,328


CALCULATING ENGAGEMENT -- (Supposedly M)

In [8]:
df2 = pd.read_csv('analysis_member_engagement.csv')
df2 = pd.DataFrame(df2)
df2['date'] = pd.to_datetime(df2['date'])
print(df2.info())
df2.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 172113 entries, 0 to 172112
Data columns (total 5 columns):
comment    172113 non-null int64
topic      172113 non-null int64
reply      172113 non-null int64
mid        172113 non-null int64
date       172113 non-null datetime64[ns]
dtypes: datetime64[ns](1), int64(4)
memory usage: 6.6 MB
None


,comment,topic,reply,mid,date
0,9,0,3,125,2018-07-15 17:00:00
1,0,0,0,210,2018-07-15 17:00:00
2,3,1,0,444,2018-07-15 17:00:00
3,0,0,0,666,2018-07-15 17:00:00
4,0,0,0,789,2018-07-15 17:00:00


In [9]:
df_en = df2.groupby(by='mid', as_index=False).sum()
df_en['score'] = df_en['comment'] + df_en['topic'] + df_en['reply']
df_en = df_en.sort_values('score', ascending=False)
df_en.head()

,mid,comment,topic,reply,score
4307,170325,574,0,639,1213
50845,3166981,889,0,0,889
20268,558560,872,8,0,880
39201,1522673,811,0,0,811
55496,3796559,553,2,235,790


In [61]:
df3 = pd.read_csv('analysis_member_score.csv')
df3 = pd.DataFrame(df3)
df3['date'] = pd.to_datetime(df3['date'])
# print(df3.info())



df3 = df3.drop('start', axis=1)
df3 = df3.drop('date', axis=1)
df3 = df3.drop('toy', axis=1)


df_en2 = df3.groupby(by='mid', as_index=False).sum()
df_en2['total'] = df_en2[df_en2.columns[1:]].sum(1)
df_en2 = df_en2[['mid', 'total']]
df_en2.head()


df_en = pd.merge(df_en,df_en2, on='mid', copy=False)
df_en['score'] = df_en['score'] + (df_en['total'] * 0.2)
df_en = df_en.drop('total',1)

df_en = df_en[['mid', 'score']]

In [32]:
df_en.head()

,mid,score
0,170325,1253.0
1,3166981,889.0
2,558560,880.0
3,1522673,811.4
4,3796559,1334.4


In [40]:
rfm = pd.merge(df_en, df_freq, on='mid')
rfm = rfm.merge(df_recency, on='mid')
rfm = rfm.rename(columns={'score':'Engagement'})
rfm = rfm[['mid', 'Recency', 'Frequency', 'Engagement']]
rfm.head()

,mid,Recency,Frequency,Engagement
0,170325,0,1925,1253.0
1,3166981,0,496,889.0
2,558560,0,486,880.0
3,1522673,0,491,811.4
4,3796559,0,1287,1334.4


In [13]:
rfm[rfm['mid']==170325]

,mid,Recency,Frequency,Engagement
0,170325,0,1925,1233.0


In [55]:
rfm['Engagement'].mean()

5.884473622232689

In [52]:
r = rfm[rfm['Engagement']== 0]
r.shape

(20358, 4)

In [41]:
quantiles = rfm.quantile(q=[0.25,0.5,0.75])
quantiles

,mid,Recency,Frequency,Engagement
0.25,472848.25,0.0,7.0,0.0
0.50,1064028.50,2.0,17.0,1.0
0.75,3181429.75,5.0,42.0,4.0


In [15]:
quantiles.to_dict()

{'mid': {0.25: 472848.25, 0.5: 1064028.5, 0.75: 3181429.75},
 'Recency': {0.25: 0.0, 0.5: 2.0, 0.75: 5.0},
 'Frequency': {0.25: 7.0, 0.5: 17.0, 0.75: 42.0},
 'Engagement': {0.25: 0.0, 0.5: 0.6000000000000001, 0.75: 2.6}}

In [16]:

# Arguments (x = value, p = recency, d = quartiles dict)
def RScore(x,p,d):
    if x <= d[p][0.25]:
        return 4
    elif x <= d[p][0.50]:
        return 3
    elif x <= d[p][0.75]: 
        return 2
    else:
        return 1
# Arguments (x = value, p = monetary_value, frequency, d = quartiles dict)
def FMScore(x,p,d):
    if x <= d[p][0.25]:
        return 1
    elif x <= d[p][0.50]:
        return 2
    elif x <= d[p][0.75]: 
        return 3
    else:
        return 4

In [95]:
rfm_segmentation = rfm
rfm_segmentation['R_Quartile'] = rfm_segmentation['Recency'].apply(RScore, args=('Recency',quantiles,))
rfm_segmentation['F_Quartile'] = rfm_segmentation['Frequency'].apply(FMScore, args=('Frequency',quantiles,))
rfm_segmentation['E_Quartile'] = rfm_segmentation['Engagement'].apply(FMScore, args=('Engagement',quantiles,))

In [94]:
rfm_segmentation['mid'] = rfm_segmentation['mid'].astype(str) 
rfm_segmentation

,mid,Recency,Frequency,Engagement,R_Quartile,F_Quartile,E_Quartile
0,170325,0,1925,1253.0,4,4,4
1,3166981,0,496,889.0,4,4,4
2,558560,0,486,880.0,4,4,4
3,1522673,0,491,811.4,4,4,4
4,3796559,0,1287,1334.4,4,4,4
5,498940,0,669,734.4,4,4,4
6,682936,0,417,644.0,4,4,4
7,160112,0,2611,653.6,4,4,4
8,63153,0,2123,622.8,4,4,4
9,649373,0,295,525.0,4,4,4


In [80]:
value = []
for item in range(1,5):
    v = rfm_segmentation[rfm_segmentation['E_Quartile'] == item].count()
    print(v)
    

mid           20358
Recency       20358
Frequency     20358
Engagement    20358
R_Quartile    20358
F_Quartile    20358
E_Quartile    20358
RFEScore      20358
dtype: int64
mid           14091
Recency       14091
Frequency     14091
Engagement    14091
R_Quartile    14091
F_Quartile    14091
E_Quartile    14091
RFEScore      14091
dtype: int64
mid           16828
Recency       16828
Frequency     16828
Engagement    16828
R_Quartile    16828
F_Quartile    16828
E_Quartile    16828
RFEScore      16828
dtype: int64
mid           16659
Recency       16659
Frequency     16659
Engagement    16659
R_Quartile    16659
F_Quartile    16659
E_Quartile    16659
RFEScore      16659
dtype: int64


In [101]:
rfm_segmentation['RFEScore'] = rfm_segmentation.R_Quartile.map(str) + rfm_segmentation.F_Quartile.map(str) + rfm_segmentation.E_Quartile.map(str)
# rfm_segmentation['Segment']

KeyError: 'Segment'

In [116]:
rfm_segmentation.head()

,mid,Recency,Frequency,Engagement,R_Quartile,F_Quartile,E_Quartile,RFEScore,Segment
0,170325,0,1925,1253.0,4,4,4,444,
1,3166981,0,496,889.0,4,4,4,444,
2,558560,0,486,880.0,4,4,4,444,
3,1522673,0,491,811.4,4,4,4,444,
4,3796559,0,1287,1334.4,4,4,4,444,


In [107]:
def segment(x):
    if x == "144":
        return 'Lost best user'
    elif x == '244':
        return 'Almost lost best user'
    elif x == '444':
        return 'Best user'
    elif x[0] == '2':
        return 'Almost lost user'
    elif x[0] == '1':
        return 'Lost user'
    elif x[2] == '4':
        return 'Big contributor'
    elif x[1] == '4':
        return 'Loyal user'
    else:
        return 'others'

In [ ]:
rfm_segmentation['Segment'] = rfm_segmentation['RFEScore'].apply(segment)

rfm_segmentation

mid      20358
score    20358
dtype: int64

In [ ]:
rfm_segmentation.to_csv('RFE_data.csv')